# D2E2S Training on Kaggle
## Using DeBERTa-v3-base (768 dims)

## Step 1: Setup Environment

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

%cd /kaggle/working
!rm -rf D2E2S
print("✅ Environment ready")

## Step 2: Clone Repository

In [ ]:
!git clone https://github.com/VishalRepos/D2E2S.git
%cd D2E2S/Codebase

print("\n" + "="*60)
print("📍 Current directory:")
!pwd

print("\n" + "="*60)
print("📝 Latest commit:")
!git log --oneline -1

print("\n" + "="*60)
print("📂 Data folder:")
!ls -la data/

print("\n" + "="*60)
print("🔍 Checking Parameter_Improved.py hidden_dim default:")
!grep -A 1 "--hidden_dim" Parameter_Improved.py

print("\n" + "="*60)

## Step 3: Install Dependencies

In [ ]:
!pip install -q torch==2.4.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q torch-geometric==2.3.1 transformers==4.28.1 tensorboardX==2.6
!pip install -q scikit-learn==1.2.2 'spacy>=3.7.2,<3.8.0' matplotlib==3.7.1
!python -m spacy download en_core_web_sm
print("✅ Dependencies installed!")

## Step 4: Verify GPU

In [ ]:
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Step 5: Run Training
### CRITICAL: hidden_dim=384 so bidirectional LSTM outputs 768

In [ ]:
print("🚀 Starting training with parameters:")
print("  - Model: microsoft/deberta-v3-base")
print("  - emb_dim: 768")
print("  - hidden_dim: 384 (bidirectional → 768)")
print("  - deberta_feature_dim: 768")
print("  - gcn_dim: 768")
print("  - mem_dim: 768")
print("="*60)

!python train.py \
    --dataset 15res \
    --batch_size 8 \
    --lr 0.0001716 \
    --epochs 100 \
    --gcn_type adaptive \
    --attention_heads 8 \
    --pretrained_deberta_name microsoft/deberta-v3-base \
    --emb_dim 768 \
    --hidden_dim 384 \
    --deberta_feature_dim 768 \
    --gcn_dim 768 \
    --mem_dim 768 \
    --sampling_processes 0 \
    --device cuda

## Step 6: View Results
### Creates interactive HTML visualization

In [ ]:
!python visualize_results.py

# Also show quick summary
import json
from pathlib import Path

results_dir = Path("data/save")
if results_dir.exists():
    runs = sorted([d for d in results_dir.iterdir() if d.is_dir()])
    if runs:
        latest = runs[-1]
        print(f"\n📊 Latest Run: {latest.name}")
        print("="*60)
        
        for file in latest.glob("*.json"):
            print(f"\n📄 {file.name}:")
            with open(file) as f:
                data = json.load(f)
                if isinstance(data, dict):
                    for k, v in data.items():
                        print(f"  {k}: {v}")
        
        html_file = latest / "results.html"
        if html_file.exists():
            print(f"\n✅ Interactive results: {html_file}")
            print("📥 Download from Output tab")